# Using the National Structure Inventory (NSI) API in Python
Author: Mark Bauer

This notebook is intended to demonstrate how to use the National Structure Inventory (NSI) API in Python. To learn more about the NSI API, please visit the [API Reference Guide](https://www.hec.usace.army.mil/confluence/nsi/technicalreferences/latest/api-reference-guide) and the [NSI Documentation](https://www.hec.usace.army.mil/confluence/nsi).

In [1]:
# import libraries
import pandas as pd

# Structures Endpoint
> The structures endpoint provides structure data to the user as a geojson feature collection (fc) or a feature stream (fs), the default is feature collection. The structure of a feature returned by the API can be seen in this geojson feature block. The type of return can be declared by the fmt directive (e.g. &fmt=fc)

Source: https://www.hec.usace.army.mil/confluence/nsi/technicalreferences/latest/api-reference-guide#id-.APIReferenceGuidev2022-Structures

For the bounding box argument, we will use the bounding box of Lower Manhattan, New York City.

![bbox](images/bbox.png)

## Structures By Bounding Box
Note: There are five points because you have to *close* the box.

In [2]:
# set variables
root_url = "https://nsi.sec.usace.army.mil/nsiapi/"
structures = "structures?bbox="
bbox = "-74.024840,40.699282,-74.024840,40.719809,-73.999305,40.699282,-73.999305,40.719809,-74.024840,40.699282"

# build full url and read as JSON in pandas
url = f"{root_url}{structures}{bbox}"
df = pd.read_json(url)

# preview data
print(df.shape)
df.head()

(6301, 2)


,type,features
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
3,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
4,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."


In [3]:
# confirm records are only feature collections
df['type'].value_counts()

FeatureCollection    6301
Name: type, dtype: int64

In [4]:
# preview feature fields
df['features'].head()

0    {'type': 'Feature', 'geometry': {'type': 'Poin...
1    {'type': 'Feature', 'geometry': {'type': 'Poin...
2    {'type': 'Feature', 'geometry': {'type': 'Poin...
3    {'type': 'Feature', 'geometry': {'type': 'Poin...
4    {'type': 'Feature', 'geometry': {'type': 'Poin...
Name: features, dtype: object

In [5]:
# normalize JSON to a flat table
df = pd.json_normalize(df['features'])

print(df.shape)
df.head()

(6301, 32)


,type,geometry.type,geometry.coordinates,properties.fd_id,properties.bid,properties.occtype,properties.st_damcat,properties.bldgtype,properties.found_type,properties.cbfips,...,properties.val_vehic,properties.source,properties.med_yr_blt,properties.firmzone,properties.o65disable,properties.u65disable,properties.x,properties.y,properties.ground_elv,properties.ground_elv_m
0,Feature,Point,"[-74.008328, 40.706836]",554665148,87G7PX4R+PMM-0-0-0-0,RES1-1SWB,RES,W,B,360610007001003,...,27000,P,1939,None,0.25,0.03,-74.008328,40.706836,21.934981,6.685782
1,Feature,Point,"[-74.018638, 40.70607]",554076983,87G7PX4J+CGH-14-14-16-12,COM8,COM,W,I,360610317044001,...,45000,P,1997,None,0.25,0.03,-74.018638,40.706070,11.275930,3.436903
2,Feature,Point,"[-74.018789, 40.705926]",554076985,87G7PX4J+9FG-7-6-7-6,COM8,COM,S,C,360610317044001,...,45000,P,1997,None,0.25,0.03,-74.018789,40.705926,11.673391,3.558049
3,Feature,Point,"[-74.018777, 40.706243]",554076986,87G7PX4J+FFX-5-5-4-5,COM8,COM,S,C,360610317044001,...,45000,P,1997,None,0.25,0.03,-74.018777,40.706243,11.516893,3.510349
4,Feature,Point,"[-74.018781, 40.706251]",554076989,87G7PX4J+GF5-1-1-2-2,COM8,COM,W,I,360610317044001,...,45000,P,1997,None,0.25,0.03,-74.018781,40.706251,11.516893,3.510349


## Structures By FIPS
FIPS codes: https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt

We will use Manhattan's FIPs code: 36063.

In [6]:
root_url = "https://nsi.sec.usace.army.mil/nsiapi/"
structures = "structures?fips="
fips = "36063"

url = f"{root_url}{structures}{fips}"
df = pd.read_json(url)

print(df.shape)
df.head()

(88613, 2)


,type,features
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
3,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
4,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."


In [7]:
df = pd.json_normalize(df['features'])

print(df.shape)
df.head()

(88613, 32)


,type,geometry.type,geometry.coordinates,properties.fd_id,properties.bid,properties.occtype,properties.st_damcat,properties.bldgtype,properties.found_type,properties.cbfips,...,properties.val_vehic,properties.source,properties.med_yr_blt,properties.firmzone,properties.o65disable,properties.u65disable,properties.x,properties.y,properties.ground_elv,properties.ground_elv_m
0,Feature,Point,"[-78.681268, 43.131174]",553845226,87M348J9+FF9-10-2-10-3,RES3A,RES,W,S,360630234051025,...,54000,X,1986,None,0.19,0.05,-78.681268,43.131174,603.945243,184.082504
1,Feature,Point,"[-78.847862, 43.243378]",553998481,87M365V2+9V2-3-1-4-1,RES3C,RES,S,B,360630243023048,...,189000,X,1967,None,0.19,0.05,-78.847862,43.243378,354.465236,108.041000
2,Feature,Point,"[-78.876219, 43.022516]",554106330,87M324FF+2G2-12-8-12-7,RES3A,RES,W,S,360630232003026,...,54000,X,1951,None,0.19,0.05,-78.876219,43.022516,575.406151,175.383789
3,Feature,Point,"[-78.964068, 43.096326]",554140975,87M332WP+G9M-2-5-3-4,RES3A,RES,M,C,360630226022005,...,54000,X,1969,None,0.19,0.05,-78.964068,43.096326,570.051757,173.751770
4,Feature,Point,"[-79.000811, 43.106657]",554249971,87M24X4X+MM8-2-5-2-5,COM1,COM,M,S,360630203001001,...,0,X,1946,None,0.19,0.05,-79.000811,43.106657,598.556458,182.440002


## Structures as Feature Collection by FIPS

In [8]:
root_url = "https://nsi.sec.usace.army.mil/nsiapi/"
structures = "structures?fips="
fips = "36063"

url = f"{root_url}{structures}{fips}&fmt=fc"
df = pd.read_json(url)

print(df.shape)
df.head()

(88613, 2)


,type,features
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
3,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
4,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poin..."


In [9]:
df = pd.json_normalize(df['features'])

print(df.shape)
df.head()

(88613, 32)


,type,geometry.type,geometry.coordinates,properties.fd_id,properties.bid,properties.occtype,properties.st_damcat,properties.bldgtype,properties.found_type,properties.cbfips,...,properties.val_vehic,properties.source,properties.med_yr_blt,properties.firmzone,properties.o65disable,properties.u65disable,properties.x,properties.y,properties.ground_elv,properties.ground_elv_m
0,Feature,Point,"[-78.681268, 43.131174]",553845226,87M348J9+FF9-10-2-10-3,RES3A,RES,W,S,360630234051025,...,54000,X,1986,None,0.19,0.05,-78.681268,43.131174,603.945243,184.082504
1,Feature,Point,"[-78.847862, 43.243378]",553998481,87M365V2+9V2-3-1-4-1,RES3C,RES,S,B,360630243023048,...,189000,X,1967,None,0.19,0.05,-78.847862,43.243378,354.465236,108.041000
2,Feature,Point,"[-78.876219, 43.022516]",554106330,87M324FF+2G2-12-8-12-7,RES3A,RES,W,S,360630232003026,...,54000,X,1951,None,0.19,0.05,-78.876219,43.022516,575.406151,175.383789
3,Feature,Point,"[-78.964068, 43.096326]",554140975,87M332WP+G9M-2-5-3-4,RES3A,RES,M,C,360630226022005,...,54000,X,1969,None,0.19,0.05,-78.964068,43.096326,570.051757,173.751770
4,Feature,Point,"[-79.000811, 43.106657]",554249971,87M24X4X+MM8-2-5-2-5,COM1,COM,M,S,360630203001001,...,0,X,1946,None,0.19,0.05,-79.000811,43.106657,598.556458,182.440002


# Stats Endpoint

> The stats endpoint provides access to a set of statistics based only on the bbox argument.

Source: https://www.hec.usace.army.mil/confluence/nsi/technicalreferences/latest/api-reference-guide#id-.APIReferenceGuidev2022-Stats

In [10]:
root_url = "https://nsi.sec.usace.army.mil/nsiapi/"
stats = "stats?bbox="
bbox = "-74.024840,40.699282,-74.024840,40.719809,-73.999305,40.699282,-73.999305,40.719809,-74.024840,40.699282"

# JSON comes as a Series, so we have to unpack the columns
url = f"{root_url}{stats}{bbox}"
df = pd.read_json(url, typ='series')

print(df.shape)
df.head()

(20,)


num_structures     6301.000000
yrbuilt_min           0.000000
yrbuilt_max        2017.000000
num_story_mean        1.468497
resunits_sum      14366.000000
dtype: float64

In [11]:
# preview full series
df

num_structures    6.301000e+03
yrbuilt_min       0.000000e+00
yrbuilt_max       2.017000e+03
num_story_mean    1.468497e+00
resunits_sum      1.436600e+04
empnum_sum        1.333500e+04
students_sum      3.038000e+03
sqft_mean         2.926842e+03
sqft_sum          1.844203e+07
pop2amu65_sum     3.000800e+04
pop2amo65_sum     5.476000e+03
pop2pmu65_sum     1.405450e+05
pop2pmo65_sum     1.227700e+04
val_struct_sum    4.282322e+09
val_cont_sum      3.107250e+09
val_vehic_sum     5.123700e+08
med_yr_blt_min    1.939000e+03
med_yr_blt_max    2.017000e+03
ground_elv_max    4.463640e+01
ground_elv_min   -5.314961e+00
dtype: float64

In [12]:
# pushing into a dataframe and reset index, rename columns
df = (
    pd.DataFrame(df)
    .reset_index()
    .rename(columns={"index":"stat", 0:"value"})
)

print(df.shape)
df

(20, 2)


,stat,value
0,num_structures,6.301000e+03
1,yrbuilt_min,0.000000e+00
2,yrbuilt_max,2.017000e+03
3,num_story_mean,1.468497e+00
4,resunits_sum,1.436600e+04
5,empnum_sum,1.333500e+04
6,students_sum,3.038000e+03
7,sqft_mean,2.926842e+03
8,sqft_sum,1.844203e+07
9,pop2amu65_sum,3.000800e+04


In [13]:
# confirm values are numeric
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   stat    20 non-null     object 
 1   value   20 non-null     float64
dtypes: float64(1), object(1)
memory usage: 448.0+ bytes
